<a href="https://colab.research.google.com/github/Melvinchen0404/Chess_engine_v2/blob/main/sapientia_v6_with_move_sounds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Initialize the environment
!pip install chess python-chess ipython datasets
!apt-get install stockfish

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.2 MB/s eta 0:00:00
  Created wheel for chess: filename=chess-1.11.1-py3-none-any.whl size=148497 sha256=6f1f76c2ccec59b96a60d92d87036edfe2ca3b28b4f573abb2fd3701ce61fdcc
  Stored in directory: /root/.cache/pip/wheels/2e/2d/23/1bfc95db984ed3ecbf6764167dc7526d0ab521cf9a9852544e
Successfully built chess
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:

In [2]:
import os
import sys
import argparse
import chess
import chess.engine
import chess.svg
from IPython.display import display, SVG, clear_output, Audio, Javascript
import ipywidgets as widgets
import threading
from openings import process_moves, uci_to_san, check_openings, get_user_moves
from gm_database import fetch_gm_data, process_gm_data
from engines import fetch_komodo_data, fetch_stockfish_data
from endgame import fetch_endgame_tablebase_data, best_endgame_move

# ANSI escape code for bold text
BOLD = "\033[1m"
RESET = "\033[0m"  # Reset formatting to normal

# Global variables
current_uci_sequence = []
current_san_sequence = []

# Sequence mode flag and moves
sequence_mode = False
sequence_moves = []
board = chess.Board()  # Initialize the board
is_initial_render = True

# Create widgets for command input and submit button
input_box = widgets.Text(placeholder='Enter your move or command', description='Command:')
submit_button = widgets.Button(description="Submit")
reset_button = widgets.Button(description="Reset")
undo_button = widgets.Button(description="Undo")
output_area = widgets.Output()  # This will hold command and analysis output
chessboard_output = widgets.Output()  # This will hold the chessboard output

# Layout the buttons horizontally
button_layout = widgets.HBox([submit_button, reset_button, undo_button])

submit_button.layout.width = reset_button.layout.width = undo_button.layout.width = '7%'
submit_button.layout.height = reset_button.layout.height = undo_button.layout.height = '20px'
submit_button.style.font = reset_button.style.font = undo_button.style.font = 'Arial 5pt'
submit_button.style.font_weight = reset_button.style.font_weight = undo_button.style.font_weight = 'normal'

layout_container = widgets.VBox([input_box, button_layout, chessboard_output, output_area])

# Function to play sound asynchronously to avoid blocking
def play_sound_async(file_path, hide_controls=True):
    try:
        display(Audio(file_path, autoplay=True, embed=True))
        if hide_controls:
            display(Javascript("""  // Hide the audio player interface
                var audio = document.querySelector('audio');
                if (audio) {
                    audio.style.display = 'none';  // Hide the audio player interface
                    audio.play();  // Ensure the audio is playing
                }
            """))
    except Exception as e:
        print(f"Error playing sound: {e}")

# Function to render the chessboard only (without regeneration)
def render_board(board, scale=0.6, game_start_sound=None, move_sound=None, hide_controls=True):
    global is_initial_render
    with chessboard_output:
        clear_output(wait=True)  # Clear the chessboard output area only
        board_svg = chess.svg.board(board, size=400 * scale)
        display(SVG(board_svg))  # Directly display the board as SVG

        if is_initial_render and game_start_sound:
            play_sound_async(game_start_sound, hide_controls=hide_controls)
            is_initial_render = False
        elif move_sound:
            play_sound_async(move_sound, hide_controls=hide_controls)

# Function to process user commands (reset, undo, etc.)
def process_command(depth, msg):
    global board, sequence_mode, sequence_moves, current_uci_sequence, current_san_sequence
    msg = msg.strip()

    current_uci_sequence = [move.uci() for move in board.move_stack]
    current_san_sequence = uci_to_san(current_uci_sequence)  # Convert UCI to SAN

    if sequence_mode:
        sequence_moves.append(msg)
        print(f"Moves in sequence: {sequence_moves}")
        if len(sequence_moves) > 0:
            for move_uci in sequence_moves:
                try:
                    process_moves(move_uci, board)
                    render_board(board, move_sound="/content/sample_data/sounds/move.mp3")

                    gm_data = fetch_gm_data(board.fen())
                    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

                    endgame_data = fetch_endgame_tablebase_data(board.fen())
                    wdl = endgame_data.get("WDL", "N/A")
                    dtz = endgame_data.get("DTZ", "N/A")

                    best_move_stockfish = fetch_stockfish_data(board.fen(), depth=8) or "No best move available"
                    best_move_komodo = fetch_komodo_data(board.fen(), depth=8) or "No best move available"
                    best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

                    opening_matches = check_openings(get_user_moves(board))

                    if opening_matches:
                        for opening_name, opening_uci in opening_matches:
                            print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

                    display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

                except ValueError as e:
                    print(f"Invalid move format: {move_uci}. Error: {e}")
            sequence_moves = []
            sequence_mode = False
    elif msg == "quit":
        close_engine()
        sys.exit()
    elif msg == "reset":
        board.reset()
        render_board(board, game_start_sound="/content/sample_data/sounds/game_start.mp3")
        fetch_and_process(board.fen())
        current_uci_sequence = []
        current_san_sequence = []
    elif msg == "undo":
        if len(board.move_stack) > 0:
            board.pop()
        render_board(board)
        fetch_and_process(board.fen())
        current_uci_sequence = [move.uci() for move in board.move_stack]
        current_san_sequence = uci_to_san(current_uci_sequence)
    elif msg.strip().lower() == "henrik":
        print("♚")
    elif msg.strip().lower() == "barbora":
        print("♛")
    elif msg == "sequence":
        print("Entering sequence mode. Enter your moves (space-separated UCI moves. For instance, 'e2e4 e7e5 g1f3'): ")
        sequence_mode = True
    else:
        try:
            process_moves(msg, board)
            render_board(board, move_sound="/content/sample_data/sounds/move.mp3")

            current_uci_sequence = [move.uci() for move in board.move_stack]
            current_san_sequence = uci_to_san(current_uci_sequence)

            gm_data = fetch_gm_data(board.fen())
            best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

            endgame_data = fetch_endgame_tablebase_data(board.fen())
            wdl = endgame_data.get("WDL", "N/A")
            dtz = endgame_data.get("DTZ", "N/A")

            best_move_stockfish = fetch_stockfish_data(board.fen(), depth=8) or "No best move available"
            best_move_komodo = fetch_komodo_data(board.fen(), depth=8) or "No best move available"
            best_move_syzygy = best_endgame_move(board.fen(), turn="white" if board.turn else "black") or "No endgame move available"

            opening_matches = check_openings(get_user_moves(board))

            if opening_matches:
                for opening_name, opening_uci in opening_matches:
                    print(f"Matched Opening: {opening_name} (UCI: {opening_uci})")

            display_best_moves_and_analysis(board.fen(), current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

        except ValueError as e:
            print(f"Invalid move format: {msg}. Error: {e}")

# Function to display best move recommendations and analysis in a table format
def fetch_and_process(fen):
    gm_data = fetch_gm_data(fen)
    best_move_uci, best_move_white_wins, best_move_black_wins, best_move_draws, top_games = process_gm_data(gm_data)

    best_move_stockfish = fetch_stockfish_data(fen, depth=8) or "No best move available"
    best_move_komodo = fetch_komodo_data(fen, depth=8) or "No best move available"
    best_move_syzygy = best_endgame_move(fen, turn="white" if board.turn else "black") or "No endgame move available"

    opening_matches = check_openings(get_user_moves(board))

    endgame_data = fetch_endgame_tablebase_data(fen)
    wdl = endgame_data.get("WDL", "N/A")
    dtz = endgame_data.get("DTZ", "N/A")

    display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz)

def display_best_moves_and_analysis(fen, current_uci_sequence, current_san_sequence, opening_matches, best_move_uci, top_games, best_move_stockfish, best_move_komodo, best_move_syzygy, wdl, dtz):
    table_rows = [
        ("Current FEN:", fen),
        ("Current UCI Sequence:", current_uci_sequence if current_uci_sequence else "No moves played yet"),
        ("Current SAN Sequence:", " ".join(current_san_sequence) if current_san_sequence else "No moves played yet"),
        ("Matched Opening from Lichess Openings Dataset:", opening_matches if opening_matches else "No matched opening available"),
        ("Best Move from GM Database:", best_move_uci if best_move_uci else "No best move available"),
        ("Top Games with Best Move from GM Database:", top_games if top_games else "No GM games found"),
        ("Best Move from Stockfish:", best_move_stockfish if best_move_stockfish else "No best move available"),
        ("Best Move from Komodo:", best_move_komodo if best_move_komodo else "No best move available"),
        ("Best Endgame Move from Syzygy:", best_move_syzygy if best_move_syzygy else "No endgame move available"),
        ("WDL (Win/Draw/Loss):", wdl),
        ("DTZ (Depth to Zero):", dtz)
    ]

    table_html = "<table style='border-collapse: collapse; width: 100%; margin: 0 auto; font-size: 10px;'>"
    table_html += """
    <tr style='background-color: #f2f2f2;'>
        <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold;'>Variable</th>
        <th style='border: 0px solid black; padding: 3px 5px; text-align: left; font-weight: bold;'>Value</th>
    </tr>
    """
    for index, row in enumerate(table_rows):
        row_color = "#ffffff" if index % 2 == 0 else "#f2f2f2"
        table_html += f"""
        <tr style='background-color: {row_color};'>
            <td style='border: 0px solid black; padding: 2px 5px;'>{row[0]}</td>
            <td style='border: 0px solid black; padding: 2px 5px;'>{row[1]}</td>
        </tr>
        """
    table_html += "</table>"

    with output_area:
        clear_output(wait=True)
        display(widgets.HTML(value=table_html))

def on_button_click(b):
    msg = input_box.value
    process_command(8, msg)
    input_box.value = ""

def on_reset_button_click(b):
    global is_initial_render
    is_initial_render = True
    board.reset()
    render_board(board, game_start_sound="/content/sample_data/sounds/game_start.mp3")
    display(Javascript("""
        var audio = document.querySelector('audio');
        if (audio) {
            audio.style.display = 'none';  // Hide the audio player interface
            audio.play();  // Ensure the audio is playing
        }
    """))

submit_button.on_click(on_button_click)
reset_button.on_click(on_reset_button_click)

# Initialize board rendering
render_board(board, game_start_sound="/content/sample_data/sounds/game_start.mp3")
display(layout_container)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/3.53k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/226k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3507 [00:00<?, ? examples/s]

Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)


...

Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)
Matched Opening: Sicilian Defense: Moscow Variation (UCI: e2e4 c7c5 g1f3 d7d6 f1b5)


Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)
Matched Opening: Sicilian Defense: Moscow Variation (UCI: e2e4 c7c5 g1f3 d7d6 f1b5)


Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)
Matched Opening: Sicilian Defense: Moscow Variation (UCI: e2e4 c7c5 g1f3 d7d6 f1b5)


Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)
Matched Opening: Sicilian Defense: Moscow Variation (UCI: e2e4 c7c5 g1f3 d7d6 f1b5)


Matched Opening: Sicilian Defense: Modern Variations (UCI: e2e4 c7c5 g1f3 d7d6)
Matched Opening: Sicilian Defense: Moscow Variation (UCI: e2e4 c7c5 g1f3 d7d6 f1b5)


In [5]:
import chess
import os
from chess.engine import SimpleEngine

# Provide the correct path to the engines when opening them
stockfish_engine = SimpleEngine.popen_uci("/usr/games/stockfish")
komodo_engine = SimpleEngine.popen_uci("/content/komodo3sse42")

def close_stockfish():
    """
    Close the Stockfish engine if it is running.
    """
    global stockfish_engine
    if stockfish_engine:
        stockfish_engine.quit()
        stockfish_engine = None
        print("Stockfish engine connection closed.")
    else:
        print("Stockfish engine is not running.")

def close_komodo():
    """
    Close the Komodo engine if it is running.
    """
    global komodo_engine
    if komodo_engine:
        komodo_engine.quit()
        komodo_engine = None
        print("Komodo engine connection closed.")
    else:
        print("Komodo engine is not running.")

# Call the close functions
close_stockfish()
close_komodo()


Stockfish engine connection closed.
Komodo engine connection closed.
